In [2]:
from config import *

In [3]:
# tf.dataset return data in X, y format
resize_rescale = tf.keras.models.Sequential([
            tf.keras.layers.Resizing(224, 224, crop_to_aspect_ratio=True),
            tf.keras.layers.Rescaling(1./255)
])

# Data augment layer
data_augment = tf.keras.models.Sequential([
            tf.keras.layers.RandomFlip("horizontal_and_vertical"),
            tf.keras.layers.RandomRotation(0.2),
            tf.keras.layers.RandomContrast(factor=0.2)

]) 

def preprocess(df, shuffle=False, augment=False):
    tdf = df.map(lambda X,y: (resize_rescale(X), y))
    if shuffle:
        tdf = tdf.shuffle(1_000,seed=42)
    # Batch all dataset
    tdf = tdf.batch(18)
    #Augmentation on training set: training=True means it will on apply during training
    if augment:
        tdf = tdf.map(lambda X,y: (data_augment(X, training=True), y))
    return tdf.prefetch(1)

train_data = preprocess(train_set_raw,True,True)
test_data  = preprocess(test_set_raw)
valid_data = preprocess(valid_set_raw)